# Introductie tot Prompt Engineering
Prompt engineering is het proces van het ontwerpen en optimaliseren van prompts voor taken op het gebied van natuurlijke taalverwerking. Het omvat het kiezen van de juiste prompts, het afstemmen van hun parameters en het evalueren van hun prestaties. Prompt engineering is essentieel om hoge nauwkeurigheid en efficiëntie te behalen in NLP-modellen. In deze sectie gaan we de basisprincipes van prompt engineering verkennen met behulp van de OpenAI-modellen voor onderzoek.


### Oefening 1: Tokenisatie
Ontdek Tokenisatie met tiktoken, een snelle open-source tokenizer van OpenAI
Zie [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) voor meer voorbeelden.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Oefening 2: Controleer de Github Models Key-instelling

Voer de onderstaande code uit om te controleren of je Github Models-endpoint correct is ingesteld. De code probeert gewoon een eenvoudige prompt en controleert de completion. De invoer `oh say can you see` zou moeten worden aangevuld met iets als `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Oefening 3: Verzinsels
Ontdek wat er gebeurt als je het LLM vraagt om antwoorden te geven op een prompt over een onderwerp dat misschien niet bestaat, of over onderwerpen waar het model mogelijk niets van weet omdat ze buiten de vooraf getrainde dataset vallen (meer recent). Kijk hoe het antwoord verandert als je een andere prompt probeert, of een ander model gebruikt.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Oefening 4: Op basis van instructie
Gebruik de variabele "text" om de hoofdinhoud in te stellen 
en de variabele "prompt" om een instructie te geven die betrekking heeft op die hoofdinhoud.

Hier vragen we het model om de tekst samen te vatten voor een leerling uit groep 4


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Oefening 5: Complexe Prompt
Probeer een verzoek waarbij systeem-, gebruiker- en assistentberichten worden gebruikt
Het systeem stelt de context van de assistent in
Gebruiker- en assistentberichten zorgen voor een gesprek met meerdere beurten

Let op hoe de persoonlijkheid van de assistent in de systeemcontext op "sarcastisch" is gezet.
Probeer eens een andere persoonlijkheidscontext. Of probeer een andere reeks input/outputberichten


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Oefening: Ontdek je intuïtie
De bovenstaande voorbeelden geven je patronen die je kunt gebruiken om nieuwe prompts te maken (eenvoudig, complex, instructie enz.) - probeer andere oefeningen te bedenken om enkele van de andere ideeën te verkennen waar we het over hebben gehad, zoals voorbeelden, aanwijzingen en meer.



---

**Disclaimer**:
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u er rekening mee te houden dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor kritische informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
